## MODULES

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from scipy.stats import cauchy
from IPython.display import display, clear_output

## Functions and test

In [8]:

# Funciones para generar trayectorias
def generate_CRW(steps, speed, initial_position, cauchy_coef):
    angles = cauchy.rvs(0, cauchy_coef, size=steps)
    x = np.cos(angles) * speed
    y = np.sin(angles) * speed
    z = np.random.normal(0, speed, size=steps)  # Z con distribución normal
    changes = np.vstack([x, y, z]).T
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory

def generate_LF(steps, speed, initial_position, cauchy_coef, alpha):
    distances = np.random.pareto(alpha, size=steps) + speed
    angles = cauchy.rvs(0, cauchy_coef, size=steps)
    x = np.cos(angles) * distances
    y = np.sin(angles) * distances
    z = np.random.normal(0, speed, size=steps)  # Z con distribución normal
    changes = np.vstack([x, y, z]).T
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory

def generate_BM(steps, speed, initial_position):
    changes = np.random.normal(0, speed, size=(steps, 3))
    trajectory = np.cumsum(changes, axis=0) + initial_position
    return trajectory


## Metrics

In [9]:
# Functions to calculate metrics
def calculate_route_length(trajectory):
    distances = np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))
    total_length = np.sum(distances)
    return total_length

def calculate_mean_square_displacement(trajectory):
    displacement = trajectory - trajectory[0]
    msd = np.mean(np.sum(displacement**2, axis=1))
    return msd

## Function to plot

In [10]:
# Function to plot trajectory and metric
def plot_trajectory_and_metric(trajectory, metric_values, metric_name):
    plt.figure(figsize=(14, 6))

    # 3D Trajectory
    ax1 = plt.subplot(1, 2, 1, projection='3d')
    ax1.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2])
    ax1.set_title('3D Trajectory')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')

    # Metric in line plot
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(metric_values, label=metric_name, color='purple')
    ax2.set_title(f'{metric_name} over time')
    ax2.set_xlabel('Step')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()



## Interface and conections with the plots

In [11]:
# Function to update the plot with the selected trajectory
def update_plot():
    initial_position = np.array([initial_position_selector.value] * 3)
    
    if current_trajectory_type == 'BM':
        trajectory = generate_BM(step_selector.value, speed_selector.value, initial_position)
    elif current_trajectory_type == 'CRW':
        trajectory = generate_CRW(step_selector.value, speed_selector.value, initial_position, cauchy_coef_selector.value)
    elif current_trajectory_type == 'LF':
        trajectory = generate_LF(step_selector.value, speed_selector.value, initial_position, cauchy_coef_selector.value, alpha_selector.value)

    if metric_selector.value == 'Route Length':
        metric_values = [calculate_route_length(trajectory[:i+1]) for i in range(1, step_selector.value+1)]
    else:  # Mean Square Displacement
        metric_values = [calculate_mean_square_displacement(trajectory[:i+1]) for i in range(1, step_selector.value+1)]

    with output:
        clear_output(wait=True)
        plot_trajectory_and_metric(trajectory, metric_values, metric_selector.value)

# Button function to select the trajectory
def on_button_click(b):
    global current_trajectory_type
    current_trajectory_type = b.description
    for button in buttons:
        button.style.button_color = 'lightgray'
    b.style.button_color = 'lightblue'
    update_plot()  # Update the plot when a new trajectory type is selected

# Real-time update function when sliders move
def connect_sliders():
    step_selector.observe(lambda change: update_plot(), names='value')
    speed_selector.observe(lambda change: update_plot(), names='value')
    initial_position_selector.observe(lambda change: update_plot(), names='value')
    cauchy_coef_selector.observe(lambda change: update_plot(), names='value')
    alpha_selector.observe(lambda change: update_plot(), names='value')
    metric_selector.observe(lambda change: update_plot(), names='value')

# Interactive interface with buttons
def interface():
    global buttons, current_trajectory_type, output
    current_trajectory_type = 'BM'
    
    buttons = [widgets.Button(description='BM'), 
               widgets.Button(description='CRW'), 
               widgets.Button(description='LF')]
    
    for button in buttons:
        button.on_click(on_button_click)
        button.style.button_color = 'lightgray'
    
    buttons[0].style.button_color = 'lightblue'  # Starts with BM selected
    
    global output, step_selector, speed_selector, initial_position_selector, cauchy_coef_selector, alpha_selector, metric_selector
    output = widgets.Output()

    # Sliders and other widgets
    step_selector = widgets.IntSlider(min=10, max=1000, step=10, value=100, description='Number of Steps:', style={'description_width': 'initial'})
    speed_selector = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Speed:', style={'description_width': 'initial'})
    initial_position_selector = widgets.FloatSlider(min=-10.0, max=10.0, step=0.1, value=0.0, description='Initial Position:', style={'description_width': 'initial'})
    cauchy_coef_selector = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Cauchy Coef.:', style={'description_width': 'initial'})
    alpha_selector = widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Lévy Alpha:', style={'description_width': 'initial'})
    metric_selector = widgets.Dropdown(options=['PL', 'MSD'], description='Metric:', style={'description_width': 'initial'})

    # Connect sliders for real-time update
    connect_sliders()

    # Create layout with menu on the left and plots on the right
    controls = widgets.VBox([widgets.HBox(buttons), step_selector, speed_selector, initial_position_selector, 
                              cauchy_coef_selector, alpha_selector, metric_selector],layout=widgets.Layout(padding='100px 0px 0px 0px'))
    layout = widgets.HBox([controls, output])
    
    display(layout)

# Call the function to display the interface
interface()